**Just a starter code with Xgboost**

In [ ]:
#Importing necessary libraries
import numpy as np

import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

import xgboost as xgb

In [ ]:
# Reading and Formatting data
trainData = pd.read_csv('../input/train.csv')
testData = pd.read_csv('../input/test.csv')
print ((trainData.shape, testData.shape))
y = trainData['loss']
trainData.drop('loss', axis =1, inplace = True)
print (trainData.shape)
# Log makes the distribution more gaussian. From the discussion forums, shift of 200
# seems to be giving the best results
y = np.log(y.add(200)) 

In [ ]:
trainData = trainData.append(testData)
print (trainData.shape)
trainData.head()
trainData.drop('id', axis=1, inplace = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labCatEncode = LabelEncoder()
trainData.ix[:,0:116] = trainData.ix[:,0:116].apply(labCatEncode.fit_transform)
train = trainData.iloc[:188318]
test = trainData.iloc[188318:]

In [ ]:
# Params for xgboost : Shamelessly copied from the user Tilii's kernel
params = {}
params['booster'] = 'gbtree'
params['objective'] = "reg:linear"
params['eval_metric'] = 'mae'
params['eta'] = 0.1
params['gamma'] = 0.5290
params['min_child_weight'] = 4.2922
params['colsample_bytree'] = 0.3085
params['subsample'] = 0.9930
params['max_depth'] = 7
params['max_delta_step'] = 0
params['silent'] = 1
params['random_state'] = 1001

In [ ]:
from sklearn.cross_validation import train_test_split
Xtrain,Xval,ytrain,yval = train_test_split(train.values, y.values, test_size  = 0.3)
dTrain = xgb.DMatrix(Xtrain, label=ytrain)
dVal = xgb.DMatrix(Xval, label=yval)
dTest = xgb.DMatrix(test.values)
watchlist = [(dTrain, 'train'), (dVal, 'eval')]
clf = xgb.train(params,dTrain,1000,watchlist,early_stopping_rounds=300)

In [ ]:
Pred = pd.DataFrame()
Pred['id'] = testData['id']
Pred['loss'] = np.exp(clf.predict(dTest))
Pred['loss']  = Pred['loss'].add(-200)
Pred.to_csv('XGB_Starter.csv', index=False)